In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from scipy.stats import norm

In [2]:
class QuantFinancialAnalysis:
    def __init__(self, tickers, period='2y'):
        self.tickers = tickers
        self.period = period
        self.stock_data = {}
        self.metrics = {}
        self.ml_predictions = {}
        
    def fetch_data(self):
        """
        Fetch and prepare data with additional features
        """
        for ticker in self.tickers:
            try:
                stock = yf.Ticker(ticker)
                data = stock.history(period=self.period)
                
                # Technical indicators
                self._add_technical_indicators(data)
                
                # Add features for ML
                self._add_ml_features(data)
                
                self.stock_data[ticker] = data
                
            except Exception as e:
                print(f"Error fetching data for {ticker}: {e}")
    
    def _add_technical_indicators(self, data):
        """
        Calculate advanced technical indicators
        """
        # Basic returns and volatility
        data['Returns'] = data['Close'].pct_change()
        data['Volatility'] = data['Returns'].rolling(window=20).std() * np.sqrt(252)
        
        # Moving averages and MACD
        data['SMA_50'] = data['Close'].rolling(window=50).mean()
        data['SMA_200'] = data['Close'].rolling(window=200).mean()
        data['EMA_12'] = data['Close'].ewm(span=12).mean()
        data['EMA_26'] = data['Close'].ewm(span=26).mean()
        data['MACD'] = data['EMA_12'] - data['EMA_26']
        data['Signal_Line'] = data['MACD'].ewm(span=9).mean()
        
        # Bollinger Bands
        data['BB_middle'] = data['Close'].rolling(window=20).mean()
        bb_std = data['Close'].rolling(window=20).std()
        data['BB_upper'] = data['BB_middle'] + (bb_std * 2)
        data['BB_lower'] = data['BB_middle'] - (bb_std * 2)
        
        # RSI
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))

    def _add_ml_features(self, data):
        """
        Add features for machine learning
        """
        # Price momentum
        data['Momentum_5'] = data['Close'].pct_change(5)
        data['Momentum_21'] = data['Close'].pct_change(21)
        
        # Volume features
        data['Volume_MA_5'] = data['Volume'].rolling(window=5).mean()
        data['Volume_MA_20'] = data['Volume'].rolling(window=20).mean()
        
        # Volatility features
        data['High_Low_Ratio'] = data['High'] / data['Low']
        data['Daily_Range'] = (data['High'] - data['Low']) / data['Close']
        
        # Log returns for better statistical properties
        data['Log_Returns'] = np.log(data['Close']/data['Close'].shift(1))

    def calculate_risk_metrics(self):
        """
        Calculate advanced risk metrics
        """
        for ticker, data in self.stock_data.items():
            returns = data['Returns'].dropna()
            
            # Value at Risk (VaR)
            var_95 = np.percentile(returns, 5)
            var_99 = np.percentile(returns, 1)
            
            # Expected Shortfall (Conditional VaR)
            es_95 = returns[returns <= var_95].mean()
            
            # Sharpe Ratio (assuming risk-free rate of 2%)
            rf = 0.02
            excess_returns = returns - rf/252
            sharpe = np.sqrt(252) * excess_returns.mean() / returns.std()
            
            # Maximum Drawdown
            cum_returns = (1 + returns).cumprod()
            rolling_max = cum_returns.expanding().max()
            drawdowns = (cum_returns - rolling_max) / rolling_max
            max_drawdown = drawdowns.min()
            
            self.metrics[ticker] = {
                'var_95': var_95,
                'var_99': var_99,
                'expected_shortfall_95': es_95,
                'sharpe_ratio': sharpe,
                'max_drawdown': max_drawdown,
                'annualized_volatility': returns.std() * np.sqrt(252),
                'annualized_return': returns.mean() * 252
            }

    def train_ml_models(self):
        """
        Train ML models for price prediction
        """
        for ticker, data in self.stock_data.items():
            # Prepare features for ML
            feature_columns = ['Momentum_5', 'Momentum_21', 'Volume_MA_5', 'Volume_MA_20',
                             'High_Low_Ratio', 'Daily_Range', 'RSI', 'Volatility']
            
            # Prepare data
            X = data[feature_columns].dropna()
            y = data['Returns'].shift(-1).dropna()  # Predict next day's returns
            
            # Align data
            X = X.iloc[:-1]
            y = y.iloc[:len(X)]
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
            
            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Train Random Forest model
            rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
            rf_model.fit(X_train_scaled, y_train)
            
            # Make predictions
            rf_predictions = rf_model.predict(X_test_scaled)
            
            # Store predictions and metrics
            self.ml_predictions[ticker] = {
                'true_values': y_test,
                'predictions': rf_predictions,
                'mse': mean_squared_error(y_test, rf_predictions),
                'r2': r2_score(y_test, rf_predictions),
                'feature_importance': dict(zip(feature_columns, rf_model.feature_importances_))
            }

    def create_dashboard(self):
        """
        Create an enhanced dashboard with ML predictions and risk metrics
        """
        for ticker, data in self.stock_data.items():
            # Create figure with more subplots
            fig = make_subplots(rows=4, cols=2,
                              subplot_titles=(f'{ticker} Price and Predictions',
                                            'ML Feature Importance',
                                            'Risk Metrics',
                                            'Returns Distribution',
                                            'Technical Indicators',
                                            'Volume Analysis',
                                            'Volatility',
                                            'RSI'),
                              vertical_spacing=0.1,
                              horizontal_spacing=0.1,
                              row_heights=[0.4, 0.2, 0.2, 0.2])

            # Price and predictions
            fig.add_trace(go.Candlestick(x=data.index,
                                        open=data['Open'],
                                        high=data['High'],
                                        low=data['Low'],
                                        close=data['Close'],
                                        name='OHLC'),
                         row=1, col=1)

            # ML Feature Importance
            if ticker in self.ml_predictions:
                importance = self.ml_predictions[ticker]['feature_importance']
                fig.add_trace(go.Bar(x=list(importance.keys()),
                                   y=list(importance.values()),
                                   name='Feature Importance'),
                             row=1, col=2)

            # Risk Metrics
            risk_metrics = self.metrics[ticker]
            fig.add_trace(go.Bar(x=list(risk_metrics.keys()),
                               y=list(risk_metrics.values()),
                               name='Risk Metrics'),
                         row=2, col=1)

            # Returns Distribution
            fig.add_trace(go.Histogram(x=data['Returns'].dropna(),
                                     name='Returns Distribution',
                                     nbinsx=50),
                         row=2, col=2)

            # Technical Indicators
            fig.add_trace(go.Scatter(x=data.index, y=data['BB_middle'],
                                   name='BB Middle',
                                   line=dict(color='blue')),
                         row=3, col=1)
            fig.add_trace(go.Scatter(x=data.index, y=data['BB_upper'],
                                   name='BB Upper',
                                   line=dict(color='gray')),
                         row=3, col=1)
            fig.add_trace(go.Scatter(x=data.index, y=data['BB_lower'],
                                   name='BB Lower',
                                   line=dict(color='gray')),
                         row=3, col=1)

            # Volume Analysis
            fig.add_trace(go.Bar(x=data.index, y=data['Volume'],
                               name='Volume'),
                         row=3, col=2)

            # Volatility
            fig.add_trace(go.Scatter(x=data.index, y=data['Volatility'],
                                   name='Volatility'),
                         row=4, col=1)

            # RSI
            fig.add_trace(go.Scatter(x=data.index, y=data['RSI'],
                                   name='RSI'),
                         row=4, col=2)

            # Update layout
            fig.update_layout(
                title=f'Quantitative Analysis Dashboard - {ticker}',
                height=1500,
                showlegend=True
            )

            fig.show()


In [3]:
def main():
    # Example usage
    tickers = ['AAPL', 'MSFT', 'GOOGL']
    
    # Initialize and run analysis
    analysis = QuantFinancialAnalysis(tickers)
    
    print("Fetching and preparing data...")
    analysis.fetch_data()
    
    print("Calculating risk metrics...")
    analysis.calculate_risk_metrics()
    
    print("Training ML models...")
    analysis.train_ml_models()
    
    # Print summary
    for ticker in tickers:
        print(f"\n=== {ticker} Analysis ===")
        print("\nRisk Metrics:")
        for metric, value in analysis.metrics[ticker].items():
            print(f"{metric}: {value:.4f}")
            
        if ticker in analysis.ml_predictions:
            print("\nML Model Performance:")
            print(f"MSE: {analysis.ml_predictions[ticker]['mse']:.6f}")
            print(f"R2 Score: {analysis.ml_predictions[ticker]['r2']:.4f}")
            
            print("\nTop Features by Importance:")
            sorted_features = sorted(
                analysis.ml_predictions[ticker]['feature_importance'].items(),
                key=lambda x: x[1],
                reverse=True
            )
            for feature, importance in sorted_features[:5]:
                print(f"{feature}: {importance:.4f}")
    
    print("\nGenerating interactive dashboards...")
    analysis.create_dashboard()

In [4]:
if __name__ == "__main__":
    main()

Fetching and preparing data...
Calculating risk metrics...
Training ML models...

=== AAPL Analysis ===

Risk Metrics:
var_95: -0.0219
var_99: -0.0358
expected_shortfall_95: -0.0307
sharpe_ratio: 0.9681
max_drawdown: -0.1736
annualized_volatility: 0.2266
annualized_return: 0.2394

ML Model Performance:
MSE: 0.000236
R2 Score: 0.0641

Top Features by Importance:
Momentum_21: 0.2246
RSI: 0.1653
Volatility: 0.1422
Volume_MA_20: 0.1186
Momentum_5: 0.1169

=== MSFT Analysis ===

Risk Metrics:
var_95: -0.0231
var_99: -0.0338
expected_shortfall_95: -0.0305
sharpe_ratio: 1.2549
max_drawdown: -0.1549
annualized_volatility: 0.2328
annualized_return: 0.3122

ML Model Performance:
MSE: 0.000144
R2 Score: 0.0285

Top Features by Importance:
Momentum_21: 0.2457
RSI: 0.1851
Momentum_5: 0.1421
Volume_MA_20: 0.1107
Volume_MA_5: 0.1020

=== GOOGL Analysis ===

Risk Metrics:
var_95: -0.0251
var_99: -0.0444
expected_shortfall_95: -0.0396
sharpe_ratio: 1.1810
max_drawdown: -0.2214
annualized_volatility: 0.